In [0]:
# !pip install tensorflow-gpu

In [0]:
!unzip /content/drive-download-20200428T190033Z-001.zip

Archive:  /content/drive-download-20200428T190033Z-001.zip
  inflating: ValidationOnly.tsv      
  inflating: Ty.txt                  
  inflating: FinalTest.tsv           
  inflating: FinalTrainingOnly.tsv   


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Embedding ,Bidirectional
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import accuracy_score

In [0]:
d = pd.read_csv('/content/FinalTrainingOnly.tsv', sep="\t", header=None )

x=d[1]
y=d[2]
y=list(y)
x=list(x)

for i in range(len(x)):
  if type(x[i])==float:
    x[i]=" "
    y[i]= 1

y=np.array(y)

In [0]:
v = pd.read_csv('/content/ValidationOnly.tsv', sep="\t", header=None )

vx=v[1]
vy=v[2]
vy=list(vy)
vx=list(vx)

for i in range(len(vx)):
  if type(vx[i])==float:
    vx[i]=" "
    vy[i]= 1

vy=np.array(vy)

In [0]:
t = pd.read_csv('/content/FinalTest.tsv', sep="\t", header=None )
tx=list(t[1])
for i in range(len(tx)):
  if type(tx[i])==float:
    tx[i]=" "

In [0]:
ty = pd.read_csv('/content/Ty.txt', sep=",", header=None )
sentiment = []
for i in range(len(ty)):
    if ty[1][i]=='negative':
        sentiment.append(0)
    elif ty[1][i]=='neutral':
        sentiment.append(1)
    elif ty[1][i]=='positive':
        sentiment.append(2)

ty = np.array(sentiment)

In [0]:
tokenizer = Tokenizer(num_words=2500,split=' ')
tokenizer.fit_on_texts(x)

X= tokenizer.texts_to_sequences(x)
X = pad_sequences(X,maxlen=50)

VX= tokenizer.texts_to_sequences(vx)
VX = pad_sequences(VX,maxlen=50)


TX = tokenizer.texts_to_sequences(tx)
TX = pad_sequences(TX,maxlen=50)

In [0]:
vocab_size = 20000
embed_size = 128

In [0]:
# Model no 1 

ip = tf.keras.Input(shape=(X.shape[1]))
em = Embedding(vocab_size, embed_size,trainable=True)(ip)
x = Bidirectional(LSTM(128,activation='tanh', return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(em) # Dimn shd be (None,200,128)
x = tf.keras.layers.Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)
avg_pool = tf.keras.layers.GlobalAveragePooling1D()(x)
max_pool = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.keras.layers.concatenate([avg_pool, max_pool])
x = Dense(3, activation='softmax')(x)
bi = tf.keras.Model(inputs = ip , outputs = x)
bi.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
# tf.keras.utils.plot_model(bi, 'my_first_model.png')
history1 = bi.fit(X, y, epochs=7, batch_size=128,verbose=2,validation_data=(VX,vy))

Train on 14594 samples, validate on 3000 samples
Epoch 1/7
14594/14594 - 30s - loss: 0.8911 - accuracy: 0.5650 - val_loss: 0.8009 - val_accuracy: 0.6500
Epoch 2/7
14594/14594 - 20s - loss: 0.7813 - accuracy: 0.6551 - val_loss: 0.8402 - val_accuracy: 0.6327
Epoch 3/7
14594/14594 - 20s - loss: 0.7217 - accuracy: 0.6849 - val_loss: 0.6718 - val_accuracy: 0.7163
Epoch 4/7
14594/14594 - 20s - loss: 0.6500 - accuracy: 0.7235 - val_loss: 0.6449 - val_accuracy: 0.7380
Epoch 5/7
14594/14594 - 20s - loss: 0.5589 - accuracy: 0.7646 - val_loss: 0.5735 - val_accuracy: 0.7840
Epoch 6/7
14594/14594 - 20s - loss: 0.5037 - accuracy: 0.7945 - val_loss: 0.5113 - val_accuracy: 0.8120
Epoch 7/7
14594/14594 - 20s - loss: 0.4215 - accuracy: 0.8320 - val_loss: 0.4847 - val_accuracy: 0.8323


In [0]:
# Model no 2 

ip = tf.keras.Input(shape=(X.shape[1]))
x = Embedding(vocab_size, embed_size,trainable=True)(ip)
x = tf.keras.layers.Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.reshape(x,(-1,1,64))
x = LSTM(64,activation='tanh', dropout=0.15, recurrent_dropout=0.15)(x)
x = Dense(3, activation='softmax')(x)
cl = tf.keras.Model(inputs=ip, outputs=x)
cl.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
history = cl.fit(X, y, epochs=5, batch_size=128,verbose=2,validation_data=(VX,vy))

Train on 14594 samples, validate on 3000 samples
Epoch 1/5
14594/14594 - 6s - loss: 0.8856 - accuracy: 0.5646 - val_loss: 0.7652 - val_accuracy: 0.6597
Epoch 2/5
14594/14594 - 3s - loss: 0.7679 - accuracy: 0.6578 - val_loss: 0.7025 - val_accuracy: 0.6930
Epoch 3/5
14594/14594 - 3s - loss: 0.6914 - accuracy: 0.7021 - val_loss: 0.6386 - val_accuracy: 0.7480
Epoch 4/5
14594/14594 - 3s - loss: 0.6056 - accuracy: 0.7530 - val_loss: 0.5576 - val_accuracy: 0.7850
Epoch 5/5
14594/14594 - 3s - loss: 0.5155 - accuracy: 0.7967 - val_loss: 0.5102 - val_accuracy: 0.8183


In [0]:
# Model no 3

model = Sequential()
model.add(Embedding(vocab_size, embed_size, input_shape = (X.shape[1],)))
model.add(LSTM(units=264, activation='tanh'))
model.add(Dense(units=64,activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(units=3, activation='softmax'))

model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics = ['accuracy'])
history = model.fit(X, y, epochs=6, batch_size=128,verbose=2,validation_data=(VX,vy))

Train on 14594 samples, validate on 3000 samples
Epoch 1/6
14594/14594 - 5s - loss: 0.9166 - accuracy: 0.5431 - val_loss: 0.7952 - val_accuracy: 0.6400
Epoch 2/6
14594/14594 - 4s - loss: 0.7893 - accuracy: 0.6431 - val_loss: 0.7434 - val_accuracy: 0.6873
Epoch 3/6
14594/14594 - 4s - loss: 0.7385 - accuracy: 0.6721 - val_loss: 0.7078 - val_accuracy: 0.7050
Epoch 4/6
14594/14594 - 4s - loss: 0.6874 - accuracy: 0.7057 - val_loss: 0.6485 - val_accuracy: 0.7407
Epoch 5/6
14594/14594 - 4s - loss: 0.6413 - accuracy: 0.7269 - val_loss: 0.6193 - val_accuracy: 0.7620
Epoch 6/6
14594/14594 - 4s - loss: 0.5906 - accuracy: 0.7523 - val_loss: 0.5948 - val_accuracy: 0.7787


In [0]:
# Model no 4

ip = tf.keras.Input(shape=(X.shape[1]))
b = Embedding(vocab_size, embed_size,trainable=True)(ip)
c1 = tf.keras.layers.Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(b)
c2 = tf.keras.layers.Conv1D(64, kernel_size=4, padding='valid', kernel_initializer='glorot_uniform')(b)
c3 = tf.keras.layers.Conv1D(64, kernel_size=5, padding='valid', kernel_initializer='glorot_uniform')(b)
x = tf.keras.layers.concatenate([c1, c2, c3],axis=1)
x = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.reshape(x,(-1,1,64))
x = Dense(64,activation='tanh')(x)
x = Dense(3, activation='softmax')(x)
cnn = tf.keras.Model(inputs=ip, outputs=x)
cnn.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
cnn.summary()
history = cnn.fit(X, y, epochs=4, batch_size=128,verbose=2,validation_data=(VX,vy))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 50, 128)      2560000     input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 48, 64)       24640       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 47, 64)       32832       embedding_3[0][0]                
____________________________________________________________________________________________

In [0]:
x1 = bi.predict(TX)
x2 = cl.predict(TX)
x3 = model.predict(TX)
x4 = cnn.predict(TX)

In [0]:
x4=np.reshape(x4,(3000,3))

In [0]:
a=[]
for i in range(len(x1)):
  b=[]
  for j in range(len(x1[i])):
    z = max(x1[i][j],x2[i][j],x3[i][j],x4[i][j] )
    b.append(z)
  a.append(b)
  b=[]


In [0]:
a=np.array(a)
a=np.argmax(a,axis=1)

In [0]:
test_acc = accuracy_score(ty,a)
print("Test accuracy = " + str(test_acc) )

Test accuracy = 0.6166666666666667
